In [13]:
# NE PAS TOUCHER!!

from IPython.display import Markdown
import ipywidgets as widgets
from ipywidgets import *
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
import csv
import cufflinks as cf
import itertools
import scipy
from scipy import stats
from scipy.stats import wilcoxon
from IPython.display import clear_output
import statsmodels.api as sm
import statsmodels.formula.api as smf
# %matplotlib tk

# Read csv
os.chdir("C:/Users/paulc/Desktop/Stage/lmt-analysis-2022/LMT/scripts/Dataframes/")
df = pd.read_csv("Merge.csv")

reset_button0 = widgets.Button(description='Reset')
reset_button1 = widgets.Button(description='Reset')
# update_button0 = widgets.Button(description='Update')
# update_button1 = widgets.Button(description='Update')
animalnumber = widgets.SelectMultiple(options=['1', '2', '3', '4'], rows=4, description='Animals', disabled=False)
event_options = [
    ["Move", "Move isolated", "Rearing", "Rear isolated", "Stop isolated", "WallJump", "SAP", "Huddling", "WaterPoint"],
    ["Contact", "Oral-oral Contact", "Oral-genital Contact", "Side by side Contact",
     "Side by side Contact, opposite way", "Social approach", "Social escape",
     "Approach contact", "Approach rear", "Break contact", "Get away", "FollowZone Isolated",
     "Train2", "Group2", "Move in contact", "Rear in contact"],
    ["Group3", "Group 3 break", "Group 3 make"],
    ["Group4", "Group 4 break", "Group 4 make", "Nest3", "Nest4"]
]

date = df["Date"].unique()
injection = df["Injection"].unique()
cage = df["Cage"].unique()
night_phase = df["Night-Phase"].unique()
bins = df["Bin"].unique()
event = df["name"].unique()
stats = ['Mixed Model', 'Non-Mixed Model']

# drop1 = widgets.SelectMultiple(options=date, rows=5, description='Date', disabled=False)
drop1 = widgets.SelectMultiple(options=injection, rows=5, description='Injection', disabled=False)
# drop1 = widgets.Dropdown(options=date, value=221013, description='Date:', disabled=False)
drop2 = widgets.SelectMultiple(options=cage, rows=5, description='Cage', disabled=False)
# drop2 = widgets.Dropdown(options=cage, value='Cage1', description='Cage:', disabled=False)
# drop3 = widgets.SelectMultiple(options=event, value=['Move'], rows=5, description='Event', disabled=False)
drop3 = widgets.Dropdown(options=event, value=event[1], description='Event:', disabled=False)
# drop4 = widgets.SelectMultiple(options=night_phase, rows=5, description='Night_Phase', disabled=False, value=[night_phase[0]])
drop4 = widgets.SelectMultiple(options=night_phase, rows=5, description='Night_Phase', disabled=False)
# drop4 = widgets.Dropdown(options=night_phase, value=1, description='Night_Phase:', disabled=False)
drop0 = widgets.Dropdown(options=stats, value=stats[0], description='Statistic:', disabled=False)
drop1_bis = widgets.SelectMultiple(options=injection, rows=5, description='Injection', disabled=False)
drop2_bis = widgets.SelectMultiple(options=cage, rows=5, description='Cage', disabled=False)
drop3_bis = widgets.Dropdown(options=event, value=event[1], description='Event:', disabled=False)
drop4_bis = widgets.SelectMultiple(options=night_phase, rows=5, description='Night_Phase', disabled=False)

# update_button0.on_click(lambda b: update_options(None))
# update_button1.on_click(lambda b: update_options(None))

box1 = HBox([reset_button0, drop1, drop2, drop4, drop3])
box2 = VBox([reset_button1, drop0, drop1_bis, drop2_bis, drop4_bis, drop3])

# On crée l'onglet pour les widgets
tab0 = widgets.VBox(children=[animalnumber])
tab1 = widgets.Tab(children=[box1])
tab2 = widgets.VBox(children=[box2])
tab1.children = (tab0,) + tab1.children + tab2.children
tab1.set_title(0, 'Animals')
tab1.set_title(1, 'Plot')
tab1.set_title(2, 'Stats')

# On affiche l'onglet
display(tab1)

def update_event_options(change):
    selected_animals = [int(x) for x in animalnumber.value]
    event_list = [event_options[i-1] for i in selected_animals]
    options = list(itertools.chain(*event_list))
    drop3.options = options
    drop3.value = options[0]

animalnumber.observe(update_event_options, names='value')

def reset_selections(b):
    drop1.value = ()
    drop2.options = df['Cage'].unique()
    drop4.options = df['Night-Phase'].unique()

reset_button0.on_click(reset_selections)

def reset_selections(b):
    drop1_bis.value = ()
    drop2_bis.options = df['Cage'].unique()
    drop4_bis.options = df['Night-Phase'].unique()

reset_button1.on_click(reset_selections)

# Pour sélectionner plusieurs Dates
def update_options0(change):
    # Filtrer les données du DataFrame en fonction des valeurs sélectionnées dans drop1
    filtered_df = df[df['Injection'].isin(drop1.value)]
    
    # Mettre à jour les options de drop2 avec les valeurs uniques de la colonne "Cage" pour les données filtrées
    options = filtered_df['Cage'].unique()
    drop2.options = options
    
    # Conserver les valeurs sélectionnées dans drop2 si elles sont encore valides après la mise à jour des options
    drop2.value = list(set(drop2.value) & set(options))
    
    # Mettre à jour les options de drop4 avec les valeurs uniques de la colonne "Night-Phase" pour les données filtrées
    options = filtered_df['Night-Phase'].unique()
    drop4.options = options
    
    # Conserver les valeurs sélectionnées dans drop4 si elles sont encore valides après la mise à jour des options
    drop4.value = list(set(drop4.value) & set(options))

# Assigner la fonction update_options à l'événement "observe" de drop1
drop1.observe(update_options0, names='value')

def update_options1(change):
    # Filtrer les données du DataFrame en fonction des valeurs sélectionnées dans drop1
    filtered_df = df[df['Injection'].isin(drop1_bis.value)]
    
    # Mettre à jour les options de drop2 avec les valeurs uniques de la colonne "Cage" pour les données filtrées
    options = filtered_df['Cage'].unique()
    drop2_bis.options = options
    
    # Conserver les valeurs sélectionnées dans drop2 si elles sont encore valides après la mise à jour des options
    drop2_bis.value = list(set(drop2_bis.value) & set(options))
    
    # Mettre à jour les options de drop4 avec les valeurs uniques de la colonne "Night-Phase" pour les données filtrées
    options = filtered_df['Night-Phase'].unique()
    drop4_bis.options = options
    
    # Conserver les valeurs sélectionnées dans drop4 si elles sont encore valides après la mise à jour des options
    drop4_bis.value = list(set(drop4_bis.value) & set(options))

# Assigner la fonction update_options à l'événement "observe" de drop1
drop1_bis.observe(update_options1, names='value')

def update_temp_df(change):
    # Mise à jour de temp_df en fonction de la sélection actuelle des widgets
    global temp_df
    temp_df = df[(df["Injection"].isin((drop1.value))) & (df["Cage"].isin((drop2.value))) & (df["name"] == drop3.value) & (df["Night-Phase"].isin((drop4.value)))]

# Initialisation de temp_df en fonction de la sélection initiale des widgets
temp_df = df[(df["Injection"].isin((drop1.value))) & (df["Cage"].isin((drop2.value))) & (df["name"] == drop3.value) & (df["Night-Phase"].isin((drop4.value)))]

# Observation des widgets pour mettre à jour temp_df lorsque la sélection est modifiée
drop1.observe(update_temp_df, 'value')
drop2.observe(update_temp_df, 'value')
drop3.observe(update_temp_df, 'value')
drop4.observe(update_temp_df, 'value')

# Préparation pour le tracé
bins = temp_df['Bin'].unique()
animals = temp_df['RFidA'].unique()
nbrAnimals = len(animals)
nbrEvents = {}

# Fonction pour afficher le plot en fonction des choix de l'utilisateur
def update_dropdown(date, cage, night_phase, event, animal):
    clear_output()
    for date_val, cage_val, night_phase_val in itertools.product(date, cage, night_phase):
        # info = f"La date choisie est le {date_val}, il s'agit de la {cage_val.lower()} pour l'évènement '{drop3.value}' durant la phase {night_phase_val} de nuit !"
        display(Markdown(f"<h3>L'analyse est réalisée sur {animalnumber.value[0]} souris, l'injection choisie est '{date_val}', il s'agit de la {cage_val} pour l'évènement '{drop3.value}' durant la phase {night_phase_val} de nuit !</h3>"))
        global temp_df_loop0
        temp_df_loop0 = df[(df["Injection"] == date_val) & (df["Cage"] == cage_val) & (df["name"] == drop3.value) & (df["Night-Phase"] == night_phase_val)]
        # Créer un nouveau dataframe avec toutes les colonnes de l'ancien dataframe
        df_new = temp_df_loop0.copy()
        # Ajouter une colonne avec la somme des numberOfEvents de chaque RFidA de chaque Bin
        df_new['new_numbersOfEvents'] = df_new.groupby(['Bin', 'RFidA'])['numberOfEvents'].transform('sum')
        # Supprimer les lignes utilisées pour faire la somme
        df_new = df_new.drop_duplicates(subset=['Bin', 'RFidA'], keep='last')
        # print(df_new)
        # plt.figure(figsize=(18, 6))
        # fig, (line_ax, bar_ax) = plt.subplots(1, 2, figsize=(20,5))
        # fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), width_ratios=[1, 1])
        fig, ax1 = plt.subplots(figsize=(12,6))
        ax2 = fig.add_axes([1, 0.25, 0.25, 0.5])
        # x=temp_df_loop[temp_df_loop.RFidA.isin(temp_df_loop.RFidA.unique())]['Bin']
        # y=temp_df_loop[temp_df_loop.RFidA.isin(temp_df_loop.RFidA.unique())]["numberOfEvents"]
        sns.lineplot(df_new, y='new_numbersOfEvents', x='Bin', hue="GenoA", style='RFidA', palette={'Amphet':'red', 'NaCl':'blue'}, ax=ax1)
        # sns.lineplot(data=temp_df_loop, x='Bin', y='numberOfEvents', hue='RFidA')
        ax1.set_title(f'Numbers of {drop3.value} per bin of each mouse')
        ax1.set_ylabel(f'Numbers of {drop3.value}')
        sns.barplot(data=df_new, y='new_numbersOfEvents', x="GenoA", palette={'Amphet':'red', 'NaCl':'blue'}, ax=ax2)
        ax2.set_title("A barplot")
        plt.show()
        mean_AMPHET = temp_df_loop0[temp_df_loop["GenoA"] == 'Amphet']["numberOfEvents"]
        mean_NACL = temp_df_loop0[temp_df_loop["GenoA"] == 'NaCl']["numberOfEvents"]
        stat, pvalue = scipy.stats.ttest_ind(mean_NACL,mean_AMPHET, alternative='two-sided')
        display(Markdown(f"<h3>La statistique du t-test est de {stat} et la p-value est de {pvalue}</h3>"))
        # table = pd.concat([mean_AMPHET, mean_NACL])
        # print(table)
        # print(mean_AMPHET)
        # print(mean_NACL)
        print(stats.kruskal(mean_AMPHET, mean_NACL))
        print(stats.shapiro(mean_AMPHET))
        # wilcoxon = scipy.stats.wilcoxon(mean_AMPHET, alternative='less')
        # display(Markdown(f"<h3>La statistique du test de Wilcoxon est de {wilcoxon.statistic} et la p-value est de {wilcoxon.pvalue}</h3>"))
        # print(len(drop2.value))
    
def update_stats(date, cage, night_phase, event, animal):
    clear_output()
    dfs = []
    for date_val, cage_val, night_phase_val in itertools.product(date, cage, night_phase):
        # info = f"La date choisie est le {date_val}, il s'agit de la {cage_val.lower()} pour l'évènement '{drop3.value}' durant la phase {night_phase_val} de nuit !"
        display(Markdown(f"<h3>L'analyse est réalisée sur {animalnumber.value[0]} souris, l'injection choisie est '{date_val}', il s'agit de la {cage_val} pour l'évènement '{drop3.value}' durant la phase {night_phase_val} de nuit !</h3>"))
        global temp_df_loop1
        temp_df_loop1 = df[(df["Injection"] == date_val) & (df["Cage"] == cage_val) & (df["name"] == drop3.value) & (df["Night-Phase"] == night_phase_val)]
        # Créer un nouveau dataframe avec toutes les colonnes de l'ancien dataframe
        df_new = temp_df_loop1.copy()
        # Ajouter une colonne avec la somme des numberOfEvents de chaque RFidA de chaque Bin
        df_new['new_numbersOfEvents'] = df_new.groupby(['Bin', 'RFidA'])['numberOfEvents'].transform('sum')
        # Supprimer les lignes utilisées pour faire la somme
        df_new = df_new.drop_duplicates(subset=['Bin', 'RFidA'], keep='last')
        # print(df_new)
        # plt.figure(figsize=(18, 6))
        # fig, (line_ax, bar_ax) = plt.subplots(1, 2, figsize=(20,5))
        # fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), width_ratios=[1, 1])
        fig, ax1 = plt.subplots(figsize=(12,6))
        ax2 = fig.add_axes([1, 0.25, 0.25, 0.5])
        # x=temp_df_loop[temp_df_loop.RFidA.isin(temp_df_loop.RFidA.unique())]['Bin']
        # y=temp_df_loop[temp_df_loop.RFidA.isin(temp_df_loop.RFidA.unique())]["numberOfEvents"]
        # sns.lineplot(df_new, y='new_numbersOfEvents', x='Bin', hue="GenoA", style='RFidA', palette={'Amphet':'red', 'NaCl':'blue'}, ax=ax1)
        # # sns.lineplot(data=temp_df_loop, x='Bin', y='numberOfEvents', hue='RFidA')
        # ax1.set_title(f'Numbers of {drop3.value} per bin of each mouse')
        # ax1.set_ylabel(f'Numbers of {drop3.value}')
        sns.barplot(data=df_new, y='new_numbersOfEvents', x="GenoA", palette={'Amphet':'red', 'NaCl':'blue'}, ax=ax2)
        ax2.set_title("A barplot")
        plt.show()
        # mean_AMPHET = temp_df_loop1[temp_df_loop["GenoA"] == 'Amphet']["numberOfEvents"]
        # mean_NACL = temp_df_loop1[temp_df_loop["GenoA"] == 'NaCl']["numberOfEvents"]
        # stat, pvalue = scipy.stats.ttest_ind(mean_NACL,mean_AMPHET, alternative='two-sided')
        # display(Markdown(f"<h3>La statistique du t-test est de {stat} et la p-value est de {pvalue}</h3>"))
        # table = pd.concat([mean_AMPHET, mean_NACL])
        # print(table)
        # print(mean_AMPHET)
        # print(mean_NACL)
        # print(stats.kruskal(mean_AMPHET, mean_NACL))
        # print(stats.shapiro(mean_AMPHET))
        # wilcoxon = scipy.stats.wilcoxon(mean_AMPHET, alternative='less')
        # display(Markdown(f"<h3>La statistique du test de Wilcoxon est de {wilcoxon.statistic} et la p-value est de {wilcoxon.pvalue}</h3>"))
        # print(len(drop2.value))
        dfs.append(temp_df_loop1)
    merged_df = pd.concat(dfs)
    model = smf.mixedlm("new_numbersOfEvents ~ GenoA", dfs, groups = 'Cage')  #Creates the model 
    result = model.fit()  #Run model
    print(result.summary())  #Print summary
    display(merged_df)
    
interactive_plot = widgets.interactive_output(update_dropdown, {'date': drop1, 'cage': drop2, 'night_phase': drop4, 'event': drop3, 'animal': animalnumber})
display(interactive_plot)

interactive_stats = widgets.interactive_output(update_stats, {'date': drop1_bis, 'cage': drop2_bis, 'night_phase': drop4_bis, 'event': drop3, 'animal': animalnumber})
display(interactive_stats)
# Faire un on_change de Date quand on clique sur la Cage après avoir choisi la Date 

Output()

Output()